# TFM Suministro Global - CREAR RELACIONES DE ETAPAS SUMINISTRO: 
##    > CREAR relación  PAIS--[REFINA] -->  MaterialREF
## _Neo4j - py2neo_
**OBSERVACIONES**  
Ya creados los nodos de PAIS y de MaterialREF


## Partimos de la existencia, en paralelo a esta ejecución, de una instancia levantada y accesible de Neo4j  
### La primera vez que se quiera usar _py2neo_ erá  necesario asegurar su instalación:  _!pip install py2neo_

In [1]:
##!pip install py2neo 

In [1]:
#importar la librería para poder hacer uso de sus funcionalidades y objetos
from py2neo import *

In [2]:
#acceder a la instancia levantada de Neo4j - en nuestro caso instancia Docker - asignamos credenciales de autentificación -
graph_test01 = Graph(auth=('neo4j', 's3cr3t'))

In [3]:
#que es el objeto - graph_test01
graph_test01

Graph('bolt://localhost:7687')

In [4]:
#que es el objeto - tx
# tx = graph_test01.begin()

In [5]:
# tx

In [6]:
#query de conexión de prueba sin errores
# verificar existencia de NODOS previos creados
#--- cuenta los nodos con diferentes tipos de etiqueta y los une en una única salida, 1 fila por cada tipo (name, cnt)
queryPRB= "MATCH (n)                                    \
RETURN labels (n)[0] as name, count (n) as Ntotal       \
UNION MATCH (n)                                         \
WITH labels (n)[1] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN name, Ntotal                                     \
UNION MATCH (n)                                         \
WITH labels (n)[2] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN name, Ntotal                                     \
UNION MATCH (n)                                         \
WITH labels (n)[3] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN name, Ntotal                                     \
"
graph_test01.run(queryPRB).to_data_frame()

,name,Ntotal
0,MaterialB,78
1,MaterialREF,78
2,Pais,116
3,GrupoAplicacion,88
4,SectorAplicacion,25


In [7]:
#validamos que todas las RELACIONES se han creado (hasta este paso)
queryPRB= "MATCH n=()-[r:INCLUIDO_EN]->()              \
RETURN type(r) as TipoRelacion, count (r) as Ntotal    \
UNION MATCH m=()-[p:USADO_EN]->()                      \
RETURN type(p) as TipoRelacion, count (p) as Ntotal"       
                                  
graph_test01.run(queryPRB).to_data_frame()

,TipoRelacion,Ntotal
0,INCLUIDO_EN,442
1,USADO_EN,442


## CARGAR DATOS

In [8]:
import numpy as np    #datos y operativa álgegra y matrices de n dimensiones (arrays)
import pandas as pd   #carga dataset y manipulación de datos  y dataframes

In [9]:
#DATOS: Fichero completo de ETAPAS DE SUMINISTRO DE MATERIALES BRUTOS (Etapa extracción) Y REFINADOS (Etapa refinado/procesado)
#       dentro del análisis de suministro crítico GLOBAL actual - Atendiendo al alcance del estudio EU CRM 2020
#FICHERO: GLEtapas12_cargar.csv
#CONTENIDO: Para cada Material, Etapa de producción 1-Extracción/Minería y  2-Refinado/procesado, 
#           Pais qye produce ese material en cada etapa que aplique, cuota de producción 
#           de ese pais y material en esa etapa e índice de trade entre 0,80 y 1,78 que da idea de la facilidad de trading  
#           para ese material en esa etapa y para ese país productor
#
#USO: Crear las RELACIONES entre "Pais" --[REFINA]--> "MaterialREF"
#     ** Propiedades de [REFINA]: 
#           SHR_PaisMat: proporción decimal de 2 dígitos (entre 0 y 1) de la cuota de producción 
#                        de ese pais para ese material en esa etapa de producción (identificada por la relación)  
#           t_PaisMat: indice de trading de 2 decimales entre 0,80 y 1,78 de esa producción,
#                      para ese pais ,  ese material y  esa etapa de producción (identificada por la relación)
#
# Ruta del fichero al Github del proyecto - TFM
rutaFichero =  "https://raw.githubusercontent.com/JuanKFD/UNIR_TFM/main/GLEtapas12_cargar.csv" 
dfEtapas12 = pd.read_csv(rutaFichero, sep=';')


In [10]:
dfEtapas12.head()

,Material,Pais1,Share1,WGI1,t1,MatRefinado,Pais2,Share2,WGI2,t2
0,Aluminium,na,NaN,NaN,NaN,Aluminium,China,"0,52","5,83","1,10"
1,Aluminium,na,NaN,NaN,NaN,Aluminium,Russian Federation,"0,06","6,20","1,10"
2,Aluminium,na,NaN,NaN,NaN,Aluminium,Other non EU,"0,08","0,00","1,00"
3,Aluminium,na,NaN,NaN,NaN,Aluminium,Canada,"0,05","2,26","1,00"
4,Aluminium,na,NaN,NaN,NaN,Aluminium,United Arab Emirates,"0,04","3,94","1,00"


In [11]:
dfEtapas12.tail()

,Material,Pais1,Share1,WGI1,t1,MatRefinado,Pais2,Share2,WGI2,t2
1126,Zirconium,na,NaN,NaN,NaN,Zirconium,na,NaN,NaN,NaN
1127,Zirconium,na,NaN,NaN,NaN,Zirconium,na,NaN,NaN,NaN
1128,Zirconium,na,NaN,NaN,NaN,Zirconium,na,NaN,NaN,NaN
1129,Zirconium,na,NaN,NaN,NaN,Zirconium,na,NaN,NaN,NaN
1130,Zirconium,na,NaN,NaN,NaN,Zirconium,na,NaN,NaN,NaN


###  IMPORTANTE .-   
En este dataset, tanto en la etapa 1 (etiquetas terminadas en 1) de producción como en la etapa 2 (etiquetas terminadas en 2)  
existen valores de Pais1 o Pais2 con el valor "na" (no aplica) . Esto indica que no debe considerarse esa línea de etapa
Así, cuando un Pais sea "na" se pasará de inmediato a procesar la siguiente línea de esa etapa.  
**El procesado de este dataset** se realizará por separado para cada etapa:
etapa 1-Extracción/Minería genera las **relaciones de tipo ("Extrae")** y para cada Pais genera un Material Bruto (MaterialB) y
etapa 2-Refinado/Procesado genera las **relaciones de tipo ("Refina")** y para cada Pais genera un Material Refinado (MaterialREF)

In [12]:
dfEtapas12.shape

(1131, 10)

In [13]:
dfEtapas12.columns

Index(['Material', 'Pais1', 'Share1', 'WGI1', 't1', 'MatRefinado', 'Pais2',
       'Share2', 'WGI2', 't2'],
      dtype='object')

**Cargados en memoria las 1131 líneas** de relación **para cada etapa**, con  _pais productor, material producido, cuota de ese país en esa producción e índice de trading t para esa etapa productiva-pais-material, atendiendo a los datos de base de partida del análisis de **cadena de suministro global** (EU CRM 2020)

# PREVISUALIZACION DE DATOS - USOS

In [15]:
type(float(3.0))

float

### Con el fin de procesar de manera más parametrizada y como presentación de otra manera de tratarlo, creo bucle en base a los propios datos de las columnas

In [17]:
## en caso de que un valor sea float, se transforma a cadena para la rápida visualización concatenada como string
strSeparator = "|"
for i in range(0, dfEtapas12.shape[0]):
    strVar = ""
    for j in range(0, dfEtapas12.shape[1]):
        if type(dfEtapas12[dfEtapas12.columns[j]][i]) == float: 
            strAux= str(dfEtapas12[dfEtapas12.columns[j]][i])
        else:
            strAux= dfEtapas12[dfEtapas12.columns[j]][i]
        strVar += strAux
        strVar += strSeparator
    print(strVar) 
# se visualiza con el separador específico identificado en la variable  - strSeparator

Aluminium|na|nan|nan|nan|Aluminium|China|0,52|5,83|1,10|
Aluminium|na|nan|nan|nan|Aluminium|Russian Federation|0,06|6,20|1,10|
Aluminium|na|nan|nan|nan|Aluminium|Other non EU|0,08|0,00|1,00|
Aluminium|na|nan|nan|nan|Aluminium|Canada|0,05|2,26|1,00|
Aluminium|na|nan|nan|nan|Aluminium|United Arab Emirates|0,04|3,94|1,00|
Aluminium|na|nan|nan|nan|Aluminium|India|0,04|5,45|1,00|
Aluminium|na|nan|nan|nan|Aluminium|Australia|0,03|2,36|1,00|
Aluminium|na|nan|nan|nan|Aluminium|United States|0,03|2,92|1,00|
Aluminium|na|nan|nan|nan|Aluminium|Norway|0,02|2,03|1,00|
Aluminium|na|nan|nan|nan|Aluminium|Brazil|0,02|5,08|1,00|
Aluminium|na|nan|nan|nan|Aluminium|Bahrain|0,02|5,15|1,00|
Aluminium|na|nan|nan|nan|Aluminium|Iceland|0,01|2,52|1,00|
Aluminium|na|nan|nan|nan|Aluminium|South Africa|0,01|4,65|1,00|
Aluminium|na|nan|nan|nan|Aluminium|Saudi Arabia|0,01|5,51|1,00|
Aluminium|na|nan|nan|nan|Aluminium|Qatar|0,01|4,03|1,00|
Aluminium|na|nan|nan|nan|Aluminium|Mozambique|0,01|5,93|1,00|
Aluminium|na|na

Neodymium|China|0,86|5,83|1,59|Neodymium|na|nan|nan|nan|
Neodymium|Australia|0,06|2,36|1,00|Neodymium|na|nan|nan|nan|
Neodymium|United States|0,02|2,92|1,00|Neodymium|na|nan|nan|nan|
Neodymium|Russian Federation|0,02|6,20|1,00|Neodymium|na|nan|nan|nan|
Neodymium|India|0,01|5,45|1,00|Neodymium|na|nan|nan|nan|
Neodymium|Brazil|0,01|5,08|1,00|Neodymium|na|nan|nan|nan|
Neodymium|Thailand|0,01|5,50|1,00|Neodymium|na|nan|nan|nan|
Neodymium|Malaysia|0,01|4,39|1,00|Neodymium|na|nan|nan|nan|
Neodymium|na|nan|nan|nan|Neodymium|na|nan|nan|nan|
Nickel|Indonesia|0,18|5,47|1,18|Nickel|China|0,29|5,83|1,29|
Nickel|Philippines|0,17|5,49|1,00|Nickel|Russian Federation|0,12|6,20|1,00|
Nickel|Australia|0,11|2,36|1,00|Nickel|Japan|0,10|2,77|1,00|
Nickel|Russian Federation|0,11|6,20|1,00|Nickel|Canada|0,08|2,26|1,00|
Nickel|Canada|0,10|2,26|1,00|Nickel|Australia|0,07|2,36|1,00|
Nickel|Other non EU|0,09|0,00|1,00|Nickel|Other non EU|0,06|0,00|1,00|
Nickel|New Caledonia|0,08|3,23|1,00|Nickel|Norway|0,05|2,03

# CREAR RELACION ENTRE  PAIS y Material Procesado (MaterialREF)
### RELACIÓN: _Pais--[REFINA]--> MaterialRef_
**Observaciones:**   
(1) Nodos previamente creados    
(2) Si para una etapa E el valor de PaisE = "na", (siendo E 1 o 2), el proceso de generación avanzará a la fila siguiente 
    puesto que la línea "na" en Pais no contiene valores. Se respeta este formato para conciliar más facilmente con los datos       de origen (excel capturada a partir de los mismos datos en el PDF oficial final y abierto del trabajo Europeo CRM 2020).
(3) Las dos relaciones (las dos etapas) contienen _DOS propiedades_:  

**SHR_PaisMat**: proporción decimal de 2 dígitos (entre 0 y 1) de la cuota de producción de ese pais 
                para ese material en esa etapa de producción (identificada por la relación)  
**t_PaisMat**: indice de trading de 2 decimales entre 0,80 y 1,78 de esa producción,
                para ese pais ,  ese material y  esa etapa de producción (identificada por la relación)
relación _EXTRAE_: Etapa1 de Extracción/Minería
relación _REFINA_: Etapa2 de Refinado/Procesado del material

### Consulta patrón _CQL-Neo4j_ para crear la relación buscada
**MATCH**   
    (a:**Pais**),  
    (b:**MaterialREF**)  

**WHERE** a.name = _"Pais_N_xxx"_ **AND** b.name = _"MaterialREF_N_xxx"_  
**CREATE** (a)**-[r:REFINA {SHR_PaisMat: 0.nn, t_PaisMat: m.mm }]->**(b)  
**RETURN** type(r)

In [18]:
nFilas = dfEtapas12.shape[0]
nFilas 

1131

In [19]:
#recordar las columnas del dataset 
dfEtapas12.columns

Index(['Material', 'Pais1', 'Share1', 'WGI1', 't1', 'MatRefinado', 'Pais2',
       'Share2', 'WGI2', 't2'],
      dtype='object')

In [29]:
#  Crear RELACION --REFINA-->   Una fila por cada relación, ETAPA 2, contiene el Pais productor y el material producido, 
# MaterialREF, así como las propiedades de cuota de producción de ese país en esta etapa y el índice de trading asociado  
# (previamente creados ambos nodos) 
#  Nodos: Pais -- para ser asignado como productor de un material para la etapa a tratar
#  Nodos: MaterialREF -- para ser asignado como producido por un Pais
## PREVISAULIZAR QUERYs antes de la ejecución (visualizar 2 de muestra)
TipoNodoA = "(a:Pais)"
TipoNodoB = "(b:MaterialREF)"
TipoRelacion = " :REFINA" 
inicio = 0
nFilas = 27 
PaisNA = "na" #valor en Pais que indica saltar línea y procesar la siguiente de esa etapa
#
# --- Procesar primeras  filas nFilas vinculadas al primer material, el "Aluminio". 
#    Si es ok, seguir con el resto (1131 - 27 ) en las siguientes celdas para esta etapa.
#    Cuando el pais es "na" -no aplica esa fila- salta la línea y procesa la siguiente
#
for i in range(inicio, nFilas):
    if dfEtapas12.Pais2[i] != PaisNA:
        strQry = "MATCH "                                    \
            + TipoNodoA + ", " + TipoNodoB                   \
            + " WHERE a.name = '"                             \
            +  dfEtapas12.Pais2[i]                            \
            + "'"                                             \
            + " AND b.name = '"                                 \
            +  dfEtapas12.MatRefinado[i]                       \
            + "_REF'"                                            \
            + " CREATE (a)-[r" + TipoRelacion                   \
            + " { SHR_PaisMat: "                                  \
            + str(dfEtapas12.Share2[i]).replace(',', '.')              \
            + ", t_PaisMat: "                                      \
            + str(dfEtapas12.t2[i]).replace(',', '.')                 \
            + "} ]->(b)"                                          \
            + " RETURN type(r)"                                 
        print("QRY [" + str(i) + "]> "+ strQry)
    else:
        continue
#
# querys  visualizadas


QRY [0]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'China' AND b.name = 'Aluminium_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.52, t_PaisMat: 1.10} ]->(b) RETURN type(r)
QRY [1]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Russian Federation' AND b.name = 'Aluminium_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.06, t_PaisMat: 1.10} ]->(b) RETURN type(r)
QRY [2]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Other non EU' AND b.name = 'Aluminium_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.08, t_PaisMat: 1.00} ]->(b) RETURN type(r)
QRY [3]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Canada' AND b.name = 'Aluminium_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.05, t_PaisMat: 1.00} ]->(b) RETURN type(r)
QRY [4]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'United Arab Emirates' AND b.name = 'Aluminium_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.04, t_PaisMat: 1.00} ]->(b) RETURN type(r)
QRY [5]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'India' AND b.name = 'Aluminium_R

In [22]:
#comprobar que sigue abierta la conexión para enviar las queries a la BD 
graph_test01

Graph('bolt://localhost:7687')


# Ejecutar  creación de relaciones iniciales. Si es OK en Neo4j, lanzar el resto 

In [23]:
# Crear la RELACIÓN en la BD , una por CADA FILA ---- primeras filas ----
#relación :REFINA (COLUMNAS xxxxxx2 - ETAPA 2)
## query para crea la relación con sus propiedades 
#
for i in range(inicio, nFilas):
    if dfEtapas12.Pais2[i] != PaisNA:
        strQry = "MATCH "                                    \
            + TipoNodoA + ", " + TipoNodoB                   \
            + " WHERE a.name = '"                             \
            +  dfEtapas12.Pais2[i]                            \
            + "'"                                             \
            + " AND b.name = '"                                 \
            +  dfEtapas12.MatRefinado[i]                       \
            + "_REF'"                                            \
            + " CREATE (a)-[r" + TipoRelacion                   \
            + " { SHR_PaisMat: "                                  \
            + str(dfEtapas12.Share2[i]).replace(',', '.')              \
            + ", t_PaisMat: "                                      \
            + str(dfEtapas12.t2[i]).replace(',', '.')                 \
            + "} ]->(b)"                                          \
            + " RETURN type(r)"                                 
        #
        # ejecutar Query en BD abierta - Neo4j
        #
        res=graph_test01.run(strQry).data()
        print(res)
        #Visualizar Query ejecutada 
        print("EJECUTADA QRY [" + str(i) + "]> "+ strQry)
    else:
        continue
# elementos creados en BD

[{'type(r)': 'REFINA'}]
EJECUTADA QRY [0]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'China' AND b.name = 'Aluminium_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.52, t_PaisMat: 1.10} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [1]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Russian Federation' AND b.name = 'Aluminium_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.06, t_PaisMat: 1.10} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [2]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Other non EU' AND b.name = 'Aluminium_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.08, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [3]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Canada' AND b.name = 'Aluminium_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.05, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [4]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'United Arab Emirates' AND b.name = 'Aluminium_REF' C

In [24]:
# guardar como dataframe el resultado de buscar todas las relaciones creadas de tipo USADO_EN
# de momento sólo están creadas las 2 primeras, y etiquetamos el resultado con el nombre de la relación 
dfGraphMat= graph_test01.run("MATCH p=()-[r:REFINA]->() RETURN (p) as REFINA").data()

In [25]:
# visualizar los datos creados usando .data() perteneciente a la funcionalidad de la librería py2neo
dfGraphMat


[{'REFINA': Path(Node('Pais', Pais_iso3='CHN', name='China'), REFINA(Node('Pais', Pais_iso3='CHN', name='China'), Node('MaterialREF', Mat_Abrev='Al', Mat_Familia='General', Mat_Subfamilia='Other non-ferrous metals', name='Aluminium_REF'), SHR_PaisMat=0.52, t_PaisMat=1.1))},
 {'REFINA': Path(Node('Pais', Pais_iso3='RUS', name='Russian Federation'), REFINA(Node('Pais', Pais_iso3='RUS', name='Russian Federation'), Node('MaterialREF', Mat_Abrev='Al', Mat_Familia='General', Mat_Subfamilia='Other non-ferrous metals', name='Aluminium_REF'), SHR_PaisMat=0.06, t_PaisMat=1.1))},
 {'REFINA': Path(Node('Pais', Pais_iso3='OTHNOEU', name='Other non EU'), REFINA(Node('Pais', Pais_iso3='OTHNOEU', name='Other non EU'), Node('MaterialREF', Mat_Abrev='Al', Mat_Familia='General', Mat_Subfamilia='Other non-ferrous metals', name='Aluminium_REF'), SHR_PaisMat=0.08, t_PaisMat=1.0))},
 {'REFINA': Path(Node('Pais', Pais_iso3='CAN', name='Canada'), REFINA(Node('Pais', Pais_iso3='CAN', name='Canada'), Node('Mater

In [26]:
#Recordar las dimensiones del dataset 
dfEtapas12.shape

(1131, 10)

### Creamos el resto de las relaciones para las restantes filas  
una vez que se verifica la creación de las n primeras filas

In [27]:
# Crear la RELACIÓN en la BD , una por CADA FILA (****restantes ***) 
#relación REFINA
## query para crea la relación con sus propiedades (en este caso no aplica)
#
inicio = nFilas
nFilas = dfEtapas12.shape[0]
# Crear la RELACIÓN en la BD , una por CADA FILA ---- primeras filas ----
#relación :REFINA (COLUMNAS xxxxxx2 - ETAPA 2)
## query para crea la relación con sus propiedades 
#
for i in range(inicio, nFilas):
    if dfEtapas12.Pais2[i] != PaisNA:
        strQry = "MATCH "                                    \
            + TipoNodoA + ", " + TipoNodoB                   \
            + " WHERE a.name = '"                             \
            +  dfEtapas12.Pais2[i]                            \
            + "'"                                             \
            + " AND b.name = '"                                 \
            +  dfEtapas12.MatRefinado[i]                       \
            + "_REF'"                                            \
            + " CREATE (a)-[r" + TipoRelacion                   \
            + " { SHR_PaisMat: "                                  \
            + dfEtapas12.Share2[i].replace(',', '.')              \
            + ", t_PaisMat: "                                      \
            + dfEtapas12.t2[i].replace(',', '.')                 \
            + "} ]->(b)"                                          \
            + " RETURN type(r)"                                 
        #
        # ejecutar Query en BD abierta - Neo4j
        #
        res=graph_test01.run(strQry).data()
        print(res)
        #Visualizar Query ejecutada 
        print("EJECUTADA QRY [" + str(i) + "]> "+ strQry)
    else:
        continue
# elementos  creados

[{'type(r)': 'REFINA'}]
EJECUTADA QRY [27]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'China' AND b.name = 'Antimony_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.59, t_PaisMat: 1.10} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [28]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Belgium' AND b.name = 'Antimony_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.12, t_PaisMat: 0.80} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [29]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'France' AND b.name = 'Antimony_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.04, t_PaisMat: 0.80} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [30]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Vietnam' AND b.name = 'Antimony_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.03, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [31]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Bolivia' AND b.name = 'Antimony_REF' CREATE (a)-[r :REFINA { SHR_Pa

[{'type(r)': 'REFINA'}]
EJECUTADA QRY [127]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Chile' AND b.name = 'Borate_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.09, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [128]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Russian Federation' AND b.name = 'Borate_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.07, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [129]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Malaysia' AND b.name = 'Borate_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.04, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [130]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Peru' AND b.name = 'Borate_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.04, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [131]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Argentina' AND b.name = 'Borate_REF' CREATE (a)-[r :REFINA {

[{'type(r)': 'REFINA'}]
EJECUTADA QRY [195]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Madagascar' AND b.name = 'Cobalt_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.03, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [196]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Russian Federation' AND b.name = 'Cobalt_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.03, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [197]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Congo, D.R.' AND b.name = 'Cobalt_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.03, t_PaisMat: 1.10} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [198]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Morocco' AND b.name = 'Cobalt_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.02, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [199]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Belgium' AND b.name = 'Cobalt_REF' CREATE (a)-[r 

[{'type(r)': 'REFINA'}]
EJECUTADA QRY [336]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'China' AND b.name = 'Fluorspar_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.34, t_PaisMat: 1.10} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [337]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Mexico' AND b.name = 'Fluorspar_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.16, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [338]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Singapore' AND b.name = 'Fluorspar_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.08, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [339]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Italy' AND b.name = 'Fluorspar_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.07, t_PaisMat: 0.80} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [340]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Germany' AND b.name = 'Fluorspar_REF' CREATE (a)-[r :REFIN

[{'type(r)': 'REFINA'}]
EJECUTADA QRY [448]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Qatar' AND b.name = 'Helium_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.17, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [449]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Algeria' AND b.name = 'Helium_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.13, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [450]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Russian Federation' AND b.name = 'Helium_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.03, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [451]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Australia' AND b.name = 'Helium_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.02, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [452]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Poland' AND b.name = 'Helium_REF' CREATE (a)-[r :REFINA 

...

[{'type(r)': 'REFINA'}]
EJECUTADA QRY [626]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'United States' AND b.name = 'Manganese_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.01, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [699]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'China' AND b.name = 'Nickel_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.29, t_PaisMat: 1.29} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [700]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Russian Federation' AND b.name = 'Nickel_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.12, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [701]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Japan' AND b.name = 'Nickel_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.10, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [702]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Canada' AND b.name = 'Nickel_REF' CREATE (a)-[r :RE

[{'type(r)': 'REFINA'}]
EJECUTADA QRY [820]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Russian Federation' AND b.name = 'Rhodium_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.12, t_PaisMat: 1.10} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [821]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Zimbabwe' AND b.name = 'Rhodium_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.05, t_PaisMat: 1.10} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [822]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Canada' AND b.name = 'Rhodium_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.02, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [823]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'United States' AND b.name = 'Rhodium_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.01, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [824]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'South Africa' AND b.name = 'Ruthenium_REF' CRE

[{'type(r)': 'REFINA'}]
EJECUTADA QRY [907]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Russian Federation' AND b.name = 'Silver_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.02, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [908]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Korea, Rep.' AND b.name = 'Silver_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.02, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [909]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'United Kingdom' AND b.name = 'Silver_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.02, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [910]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Mexico' AND b.name = 'Silver_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.02, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [911]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Australia' AND b.name = 'Silver_REF' CREATE (a

[{'type(r)': 'REFINA'}]
EJECUTADA QRY [1001]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Peru' AND b.name = 'Tellurium_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.02, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [1002]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Bulgaria' AND b.name = 'Tellurium_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.01, t_PaisMat: 0.80} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [1012]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'China' AND b.name = 'Tin_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.47, t_PaisMat: 1.42} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [1013]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Indonesia' AND b.name = 'Tin_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.19, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [1014]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Malaysia' AND b.name = 'Tin_REF' CREATE (a)-[r :REFINA { SHR_Pai

[{'type(r)': 'REFINA'}]
EJECUTADA QRY [1096]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Peru' AND b.name = 'Zinc_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.03, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [1097]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Other non EU' AND b.name = 'Zinc_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.05, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [1098]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Mexico' AND b.name = 'Zinc_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.02, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [1099]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Kazakhstan' AND b.name = 'Zinc_REF' CREATE (a)-[r :REFINA { SHR_PaisMat: 0.02, t_PaisMat: 1.00} ]->(b) RETURN type(r)
[{'type(r)': 'REFINA'}]
EJECUTADA QRY [1100]> MATCH (a:Pais), (b:MaterialREF) WHERE a.name = 'Finland' AND b.name = 'Zinc_REF' CREATE (a)-[r :REFINA { SHR_PaisM

In [25]:
#validamos que todos los nodos se han creado (hasta este paso)
queryPRB= "MATCH (n)                                    \
RETURN labels (n)[0] as name, count (n) as Ntotal       \
UNION MATCH (n)                                         \
WITH labels (n)[1] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN name, Ntotal                                     \
UNION MATCH (n)                                         \
WITH labels (n)[2] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN  name, Ntotal                                    \
UNION MATCH (n)                                         \
WITH labels (n)[3] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN  name, Ntotal                                    \
UNION MATCH (n)                                         \
WITH labels (n)[4] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN  name, Ntotal"  
                                  
graph_test01.run(queryPRB).to_data_frame()

,name,Ntotal
0,MaterialB,78
1,MaterialREF,78
2,Pais,116
3,GrupoAplicacion,88
4,SectorAplicacion,25


In [28]:
#validamos que todas las RELACIONES se han creado (hasta este paso)
queryPRB= "MATCH n=()-[r:INCLUIDO_EN]->()              \
RETURN type(r) as TipoRelacion, count (r) as Ntotal    \
UNION MATCH m=()-[p:USADO_EN]->()                      \
RETURN type(p) as TipoRelacion, count (p) as Ntotal    \
UNION MATCH m=()-[p:REFINA]->()                        \
RETURN type(p) as TipoRelacion, count (p) as Ntotal"       
                                  
graph_test01.run(queryPRB).to_data_frame()

,TipoRelacion,Ntotal
0,INCLUIDO_EN,442
1,USADO_EN,442
2,REFINA,475


# fin proceso - relación  "REFINA" - CREADA

In [81]:
# fin archivo